<a href="https://colab.research.google.com/github/sebastiangrisalesm-collab/Entregas_IA/blob/main/03_modelo_con_One_Hot_y_Red_Neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 📚***Importación de librerías:***

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from google.colab import files

🌐***Descarga de los datos desde Kaggle*** :

In [ ]:

files.upload()
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
# Descargar los datos de la competencia #
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Saving kaggle.json to kaggle (1).json
udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_example.csv  
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
# Tamaño del dataset #
print("El tamaño del dataset es:", df.shape)
df.info()

El tamaño del dataset es: (692500, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROP

🧐***Imputación de valores faltantes:***

In [ ]:
# Detectar columnas por tipo #
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# Las numéricas las imputamos con mediana #
from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy='median')
df[num_cols] = num_imputer.fit_transform(df[num_cols])

# Las categóricas las imputamos con moda o "missing" #
for col in cat_cols:
    if df[col].isna().mean() > 0.1:
        df[col] = df[col].fillna("missing")
    else:
        df[col] = df[col].fillna(df[col].mode()[0])

print("Imputación de valores completada correctamente.")

Imputación de valores completada correctamente.


🔢***Codificación One-Hot:***

In [ ]:
# Seleccionar columnas categóricas con pocas categorías #
cat_cols_limpias = [col for col in cat_cols if df[col].nunique() <= 10]

df_codificado = pd.get_dummies(df, columns=cat_cols_limpias, drop_first=True)

print("Codificación One-Hot completada. Las nuevas dimensiones del dataset son:", df_codificado.shape)

Codificación One-Hot completada. Las nuevas dimensiones del dataset son: (692500, 37)


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Crear la variable objetivo (y)

target_map = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}
y = df['RENDIMIENTO_GLOBAL'].map(target_map)


# Crear las características (X)
target_ohe_cols = [col for col in df_codificado.columns if 'RENDIMIENTO_GLOBAL_' in col]

duplicate_cols = ['F_TIENEINTERNET.1_Si']


cols_to_drop = ['ID'] + target_ohe_cols + duplicate_cols


X = df_codificado.drop(columns=cols_to_drop)

# Verificar los resultados
print(f"Forma de X (características): {X.shape}")
print(f"Forma de y (objetivo): {y.shape}")

print("\n--- Objetivo (y) ---")
print(y.head())

print("\n--- Características (X) ---")
print(X.info())
print(df_codificado.isna().sum())
df_codificado.to_csv("dataset_codificado_imputado.csv", index=False)
print("Archivo CSV creado correctamente.")
from google.colab import files
files.download("dataset_codificado_imputado.csv")



Forma de X (características): (692500, 32)
Forma de y (objetivo): (692500,)

--- Objetivo (y) ---
0    2
1    0
2    0
3    3
4    1
Name: RENDIMIENTO_GLOBAL, dtype: int64

--- Características (X) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 32 columns):
 #   Column                                                                Non-Null Count   Dtype  
---  ------                                                                --------------   -----  
 0   PERIODO_ACADEMICO                                                     692500 non-null  float64
 1   E_PRGM_ACADEMICO                                                      692500 non-null  object 
 2   E_PRGM_DEPARTAMENTO                                                   692500 non-null  object 
 3   F_EDUCACIONPADRE                                                      692500 non-null  object 
 4   F_EDUCACIONMADRE                                                      692500 non-nu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install category_encoders
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from category_encoders import TargetEncoder

# Definir X e Y

target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
y = df['RENDIMIENTO_GLOBAL'].map(target_map)

# Columnas a eliminar de las características
target_ohe_cols = [col for col in df_codificado.columns if 'RENDIMIENTO_GLOBAL_' in col]
duplicate_cols = ['F_TIENEINTERNET.1_Si']
cols_to_drop = ['ID'] + target_ohe_cols + duplicate_cols
X = df_codificado.drop(columns=cols_to_drop)

# Identificar Tipos de Columnas en X
high_card_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = df.select_dtypes(include=['float64', 'int64']).drop(columns=['ID', 'PERIODO_ACADEMICO']).columns.tolist()
ohe_cols = X.select_dtypes(include=['bool']).columns.tolist()

# Definir el Preprocesador
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('target_encoder', TargetEncoder(handle_missing='value', handle_unknown='value'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, high_card_cols),
        ('ohe', 'passthrough', ohe_cols)
    ],
    remainder='drop'
)

# Dividir y Preprocesar los datos


# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Preprocesando datos para la Red Neuronal...")
# Ajustar el preprocesador en X_train y transformar X_train
X_train_processed = preprocessor.fit_transform(X_train, y_train)
# Transformar X_test usando el preprocesador ya ajustado
X_test_processed = preprocessor.transform(X_test)

# Convertir 'y' a One-Hot Encoding para Keras (para 4 clases)
y_train_ohe = to_categorical(y_train, num_classes=4)
y_test_ohe = to_categorical(y_test, num_classes=4)

print(f"Forma de X_train procesado: {X_train_processed.shape}")
print(f"Forma de y_train OHE: {y_train_ohe.shape}")

# Importar el Callback de EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

# Importar el Callback de EarlyStopping y BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

# --- Construir y Entrenar la Red Neuronal (Keras) --- #

n_features = X_train_processed.shape[1]

model_nn = Sequential([

    Dense(256, activation='relu', input_shape=(n_features,)),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(4, activation='softmax')
])

# Compilar el modelo
model_nn.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

# Definir el EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

print("\nEntrenando modelo de Red Neuronal (Arquitectura Mejorada)...")

# Entrenar el modelo
history = model_nn.fit(
    X_train_processed,
    y_train_ohe,
    epochs=100,
    batch_size=64, #
    validation_split=0.2,
    callbacks=[early_stopping]
)

# --- Evaluar el Modelo ---
print("\nEvaluando modelo...")
loss, acc_nn = model_nn.evaluate(X_test_processed, y_test_ohe, verbose=0)

print(f"\n--- ¡Entrenamiento de Red Neuronal completado! ---")
print(f"La precisión (Accuracy) del modelo NN en los datos de prueba es: {acc_nn*100:.2f}%")

Preprocesando datos para la Red Neuronal...
Forma de X_train procesado: (554000, 31)
Forma de y_train OHE: (554000, 4)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Entrenando modelo de Red Neuronal (Arquitectura Mejorada)...
Epoch 1/100
6925/6925 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.3985 - loss: 1.2801 - val_accuracy: 0.4174 - val_loss: 1.2304
Epoch 2/100
6925/6925 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.4169 - loss: 1.2283 - val_accuracy: 0.4189 - val_loss: 1.2230
Epoch 3/100
6925/6925 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.4201 - loss: 1.2261 - val_accuracy: 0.4202 - val_loss: 1.2219
Epoch 4/100
6925/6925 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.4223 - loss: 1.2224 - val_accuracy: 0.4190 - val_loss: 1.2244
Epoch 5/100
6925/6925 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.4201 - loss: 1.2237 - val_accuracy: 0.4205 - val_loss: 1.2199
Epoch 6/100
6925/6925 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.4227 - loss: 1.2221 - val_accuracy: 0.4205 - val_loss: 1.2195
Epoch 7/100
6925/6925 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.4213 - loss: 1.2224 - val_accuracy: 0.4189 - val_loss: 1.2200
Epoch 8/100
69